In [39]:
# https://github.com/swar/nba_api/issues/124

In [40]:
import pandas as pd
from time import time
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import playerawards

In [41]:
from nba_api.stats.library import http

print(http.STATS_HEADERS)

{'Host': 'stats.nba.com', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0', 'Accept': 'application/json, text/plain, */*', 'Accept-Language': 'en-US,en;q=0.5', 'Accept-Encoding': 'gzip, deflate, br', 'x-nba-stats-origin': 'stats', 'x-nba-stats-token': 'true', 'Connection': 'keep-alive', 'Referer': 'https://stats.nba.com/', 'Pragma': 'no-cache', 'Cache-Control': 'no-cache'}


In [42]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [43]:
# function which collects player seasons and drops before modern 3 point era and insufficient data

def collect_seasons(id):
    season_filter = 1998 # the first year of the modern 3 point era

    career = playercareerstats.PlayerCareerStats(player_id=id)
    df = career.get_data_frames()[0]

    df['SEASON_ID'] = df['SEASON_ID'].apply(lambda x: int(x[:4]) + 1) #make seasons numeric type
    df = df[df['SEASON_ID'] >= season_filter] # drop any seasons before modern era

    return df

In [44]:
# function which gets player awards

# fun fact and test case: dominique wilkins in 94 made an All-NBA team
# but played for two teams that year

def collect_awards(id):
    awards = playerawards.PlayerAwards(player_id=id)

    df = awards.get_data_frames()[0]

    df = df[df['DESCRIPTION'] == 'All-NBA']
    df = df[['SEASON', 'ALL_NBA_TEAM_NUMBER']]

    df['SEASON'] = df['SEASON'].apply(lambda x: int(x[:4]) + 1) #make seasons numeric type

    return df

In [45]:
# function which joins the awards and seasons

def stat_join(seasons, awards):
    df = seasons.join(awards.set_index('SEASON'), on='SEASON_ID')
    df['ALL_NBA_TEAM_NUMBER'] = df['ALL_NBA_TEAM_NUMBER'].fillna(0)
    return df

In [46]:
# function which loops through each player

nba_players = players.get_players()

In [47]:
len(nba_players)

4900

In [48]:
nba_players[0]

{'id': 76001,
 'full_name': 'Alaa Abdelnaby',
 'first_name': 'Alaa',
 'last_name': 'Abdelnaby',
 'is_active': False}

In [49]:
len(nba_players[4892: 4900])

8

In [52]:
def collect_chunk(nba_players, start_point, chunk_size):
    print(f'start point = {start_point}')
    final_df = pd.DataFrame()
    for count, player in enumerate(nba_players[start_point:start_point + chunk_size]):
        id = player['id']
        name = player['full_name']

        start = time()
        print(f"Processing {name}...")

        seasons = collect_seasons(id)
        awards = collect_awards(id)
        df = stat_join(seasons, awards)
        df['NAME'] = player['full_name']

        if count == 0:
            final_df = df
        else:
            final_df = pd.concat([final_df, df])
        
        end = time()
        print(f"Took {end - start} seconds to finish count number {count}")

    return final_df

In [54]:
chunk_size = 100
start_point = 3300

while start_point < len(nba_players):
    if start_point + chunk_size > len(nba_players):
        chunk_size = len(nba_players) - start_point
    print(f'collecting chunk {start_point} through {start_point + chunk_size}')
    df = collect_chunk(nba_players, start_point, chunk_size)
    df.to_csv(f'../data/chunks/players_{start_point}_to_{start_point + chunk_size}.csv')
    start_point = start_point + chunk_size

collecting chunk 2100 through 2200
start point = 2100
Processing Winfred Jacobs...
Took 0.6909525394439697 seconds to finish count number 0
Processing Casey Jacobsen...


/tmp/ipykernel_4467/1331416308.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, df])


Took 0.5616276264190674 seconds to finish count number 1
Processing Sam Jacobson...
Took 0.5572407245635986 seconds to finish count number 2
Processing Dave Jamerson...
Took 0.540339469909668 seconds to finish count number 3
Processing Aaron James...


/tmp/ipykernel_4467/1331416308.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, df])


Took 0.5224177837371826 seconds to finish count number 4
Processing Bernard James...
Took 0.4731872081756592 seconds to finish count number 5
Processing Damion James...
Took 0.46660757064819336 seconds to finish count number 6
Processing Gene James...


...

/tmp/ipykernel_4467/1331416308.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, df])


Took 2.3427376747131348 seconds to finish count number 47
Processing Anthony Parker...
Took 2.7548818588256836 seconds to finish count number 48
Processing Jabari Parker...
Took 2.6046924591064453 seconds to finish count number 49
Processing Smush Parker...
Took 1.5343682765960693 seconds to finish count number 50
Processing Sonny Parker...
Took 1.519111156463623 seconds to finish count number 51
Processing Tony Parker...
Took 2.673311233520508 seconds to finish count number 52
Processing Jack Parkinson...


/tmp/ipykernel_4467/1331416308.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, df])


Took 8.369545936584473 seconds to finish count number 53
Processing Cherokee Parks...
Took 12.544695138931274 seconds to finish count number 54
Processing Jack Parr...


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)